In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import csv
import json
from keys import FMP_KEY
import os

stocks_path = '/Data/stocks.csv'

In [ ]:
def generate_dates_5_years():
    """
    Generate dates in a %Y-%m-%d format
    and scrape the data for this specific period
    """
    today = datetime.now()

    five_years_ago = today - timedelta(days=5*365)

    dates = []

    current_date = today

    # Generate dates until we reach the date 5 years ago
    while current_date >= five_years_ago:
        dates.append(current_date.strftime('%Y-%m-%d'))
        current_date -= timedelta(days=1)

    return dates, today.strftime('%Y-%m-%d'), five_years_ago.strftime('%Y-%m-%d')

In [ ]:
russels_stocks_df = pd.read_csv(stocks_path)

print(russels_stocks_df.head())

          Company Ticker  CF_ID
0     Exxon Mobil    XOM     22
1         Chevron    CVX     21
2  ConocoPhillips    COP    186
3   EOG Resources    EOG    214
4    Schlumberger    SLB    177


In [ ]:
dates_list, to_date, from_date = generate_dates_5_years()

In [ ]:
print(dates_list[:5])
print(dates_list[-5:])

['2024-06-13', '2024-06-12', '2024-06-11', '2024-06-10', '2024-06-09']
['2019-06-19', '2019-06-18', '2019-06-17', '2019-06-16', '2019-06-15']


# **Daily Prices**
The FMP Daily Chart endpoint provides a daily chart for a given company. The chart displays the opening price, high price, low price, and closing price of the company's stock for each day in a specified date range. By default the limit is 5 years of historical data, to get data prior to this date, please use the to & from parameters with a limit of 5 years.
[Read More](https://site.financialmodelingprep.com/developer/docs?gad_source=1&gclid=CjwKCAjwr7ayBhAPEiwA6EIGxBp2TkiBy3tGxuQMrZ8A-86Tso1exJ8Iu9_kMqUooBqcPByZejYh6BoCiHYQAvD_BwE#dividends-historical-dividends:~:text=1492644%0A%09%7D%0A%5D-,Daily%20Chart%20EOD%20API,-The%20FMP%20Daily)

In [ ]:
def fetch_daily_data(symbol, from_time=None, to_time=None):
    """
    Fetches the following daily metrics for each symbol (ticker):
    'open', 'changeOverTime', 'high', 'low', 'close', 'adjClose',
    'change', 'changePercent', 'vwap'
    """
    intraday_price_url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={FMP_KEY}'
    response = requests.get(intraday_price_url)

    if response.status_code == 200:
        data = response.json()
        if len(data) > 0:
            return data#[0]#['close']
    print(f'Problem occured with company: {symbol}')
    return None


def create_csv_data(data_dict, dates_list, keys_to_save):
    """
    This will create a dictionary for each field we want to save in
    a CSV file for the last 5 years on a daily basis for one stock
    """
    symbol = data_dict['symbol']
    csv_data = {}

    for key in keys_to_save:
        csv_data[key] = {}

        for item in data_dict['historical']:
            date = item['date']
            value = item.get(key, None)
            csv_data[key][date] = value
        for date in dates_list:
            if date not in csv_data[key]:
                csv_data[key][date] = None
    return csv_data

In [ ]:
directory = './Data/FMP data/Daily Prices/'

os.makedirs(directory, exist_ok=True)

In [ ]:
for symbol in list(russels_stocks_df['Ticker']):
    data = fetch_daily_data(symbol)

    if data:
      csv_data_dict = create_csv_data(data, dates_list, ['open', 'changeOverTime', 'high', 'low', 'close', 'adjClose', 'change', 'changePercent', 'vwap'])

      for key in ['open', 'changeOverTime', 'high', 'low', 'close', 'adjClose', 'change', 'changePercent', 'vwap']:
          file_path = os.path.join(directory, f"{key}.csv")

          with open(file_path, 'a') as f:
              if os.path.getsize(file_path) == 0:
                  f.write('Ticker,' + ','.join(dates_list) + '\n')

              values = []
              for date in dates_list:
                values.append(csv_data_dict[key].get(date, None))
              values_str = ','.join(map(str, values))
              f.write(f"{symbol},{values_str}\n")

In [ ]:
df = pd.read_csv('./Data/FMP data/Daily Prices/change.csv')

count = sum((df[df['Ticker'] != 'LTHM'][col].isnull().any() and not df[df['Ticker'] != 'LTHM'][col].isnull().all()) for col in df.columns)

print("Number of columns with at least one None value but not all are None:", count)

# **Technical Indicators**
Technical indicators are mathematical calculations applied to historical market data, like price and volume, to assist traders and analysts in predicting future price movements. They provide visual representations and signals, helping traders make informed decisions about buying or selling assets. However, they are based on past data, have limitations, and should be used alongside other forms of analysis for more accurate trading strategies.

[Read more](https://site.financialmodelingprep.com/developer/docs?gad_source=1&gclid=CjwKCAjwr7ayBhAPEiwA6EIGxBp2TkiBy3tGxuQMrZ8A-86Tso1exJ8Iu9_kMqUooBqcPByZejYh6BoCiHYQAvD_BwE#technical-intraday-rsi:~:text=%7D%0A%09%5D%0A%7D-,Technical%20Indicators,-Technical%20indicators%20are)

In [ ]:
def get_tecnical_indicators(symbol, indicator_type, from_date, to_date):
    """
    Fetches the following technical indicator for each symbol:
    'sma', 'ema', 'wma', 'dema', 'tema', 'williams', 'rsi', 'adx',
    'standardDeviation'
    """
    intraday_indicator_url = f'https://financialmodelingprep.com/api/v3/technical_indicator/1day/{symbol}?type={indicator_type}&period=10&from={from_date}&to={to_date}&apikey={FMP_KEY}'
    response = requests.get(intraday_indicator_url)

    if response.status_code == 200:
        data = response.json()
        if len(data) > 0:
            return data#[0]#['close']
    else:
      print(f'Problem occured with company {symbol} for type {indicator_type}')
    return None

def convert_to_date_dict(data_list, indicator):
    """
    Helper function to get the json output in the
    desired format
    """
    return {entry['date'][:10]: entry[indicator] for entry in data_list}

In [ ]:
INDICATORS = ['sma' 'ema', 'wma', 'dema', 'tema', 'williams', 'rsi', 'adx', 'standardDeviation'] # finished:
# intraday_price_url = f'https://financialmodelingprep.com/api/v3/technical_indicator/1day/AAPL?type=williams&period=10&from={from_date}&to={to_date}&apikey={FMP_KEY}'
# response = requests.get(intraday_price_url)

directory = './Data/FMP data/Commodities/Technical Indicators/'

os.makedirs(directory, exist_ok=True)

In [ ]:
for symbol in list(commodities_df['Symbol']):
    for indicator in INDICATORS:
        indicator_data = get_tecnical_indicators(symbol, indicator, from_date, to_date)
        if indicator_data:
          indicator_data_dict = convert_to_date_dict(indicator_data, indicator)

          file_path = os.path.join(directory, f"{indicator}.csv")

          if not os.path.exists(file_path):
              # If file does not exist, create it and write the header
              with open(file_path, 'w') as f:
                  f.write('Ticker,' + ','.join(dates_list) + '\n')

          # Append the data for the current symbol
          with open(file_path, 'a') as f:
              values_str = ','.join(map(str, [indicator_data_dict.get(date, None) for date in dates_list]))
              f.write(f"{symbol},{values_str}\n")
    print(f"Done company: {symbol}")

# **Historical Social Sentiment API**

In [ ]:
intraday_indicator_url = f'https://financialmodelingprep.com/api/v4/historical/social-sentiment?symbol=LCID&apikey={FMP_KEY}&from={from_date}&to={to_date}&page=150'
# intraday_indicator_url = f'https://financialmodelingprep.com/api/v3/historical-price-full/stock_split/AAPL?apikey={FMP_KEY}'
response = requests.get(intraday_indicator_url)

In [ ]:
response.json()

[{'date': '2022-05-02 15:00:00',
  'symbol': 'LCID',
  'stocktwitsPosts': 11,
  'twitterPosts': 25,
  'stocktwitsComments': 7,
  'twitterComments': 571,
  'stocktwitsLikes': 15,
  'twitterLikes': 1913,
  'stocktwitsImpressions': 7367,
  'twitterImpressions': 536238,
  'stocktwitsSentiment': 0.4901,
  'twitterSentiment': 0.5413},
 {'date': '2022-05-02 14:00:00',
  'symbol': 'LCID',
  'stocktwitsPosts': 22,
  'twitterPosts': 36,
  'stocktwitsComments': 15,
  'twitterComments': 86,
  'stocktwitsLikes': 46,
  'twitterLikes': 861,
  'stocktwitsImpressions': 130521,
  'twitterImpressions': 248299,
  'stocktwitsSentiment': 0.548,
  'twitterSentiment': 0.5582},
 {'date': '2022-05-02 13:00:00',
  'symbol': 'LCID',
  'stocktwitsPosts': 29,
  'twitterPosts': 26,
  'stocktwitsComments': 7,
  'twitterComments': 475,
  'stocktwitsLikes': 77,
  'twitterLikes': 1670,
  'stocktwitsImpressions': 1958,
  'twitterImpressions': 76261,
  'stocktwitsSentiment': 0.55,
  'twitterSentiment': 0.5759},
 {'date': 

# **Commodities**

In [ ]:
intraday_indicator_url = f'https://financialmodelingprep.com/api/v3/symbol/available-commodities?apikey={FMP_KEY}'
response = requests.get(intraday_indicator_url)

In [ ]:
response.json()

[{'symbol': 'ESUSD',
  'name': 'E-Mini S&P 500',
  'currency': 'USD',
  'stockExchange': 'CME',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'ZMUSD',
  'name': 'Soybean Meal Futures',
  'currency': 'USD',
  'stockExchange': 'CME',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'ZQUSD',
  'name': '30 Day Fed Fund Futures',
  'currency': 'USD',
  'stockExchange': 'CBOE',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'ALIUSD',
  'name': 'Aluminum Futures',
  'currency': 'USD',
  'stockExchange': 'COMEX',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'ZBUSD',
  'name': '30 Year U.S. Treasury Bond Futures',
  'currency': 'USD',
  'stockExchange': 'CME',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'GCUSD',
  'name': 'Gold Futures',
  'currency': 'USD',
  'stockExchange': 'CME',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'ZLUSX',
  'name': 'Soybean Oil Futures',
  'currency': 'USX',
  'stockExchange': 'CME',
  'exchangeShortName': 'COMMODITY'},
 {'symbol': 'KEUSX',
  'n

In [ ]:
commodities_path = './Data/commodities.csv'

In [ ]:
commodities_df = pd.read_csv(commodities_path)

print(commodities_df.head())

       Name  Symbol  CF_ID
0      Gold   GCUSD     50
1    Silver   SIUSD     61
2    Copper   HGUSD   3685
3  Platinum   PLUSD   3686
4  Aluminum  ALIUSD   5500


In [ ]:
directory = './Data/FMP data/Commodities/Daily Prices/'

os.makedirs(directory, exist_ok=True)

In [ ]:
for symbol in list(commodities_df['Symbol']):
    data = fetch_daily_data(symbol)

    if data:
      csv_data_dict = create_csv_data(data, dates_list, ['open', 'changeOverTime', 'high', 'low', 'close', 'adjClose', 'change', 'changePercent', 'vwap'])

      for key in ['open', 'changeOverTime', 'high', 'low', 'close', 'adjClose', 'change', 'changePercent', 'vwap']:
          file_path = os.path.join(directory, f"{key}.csv")

          with open(file_path, 'a') as f:
              if os.path.getsize(file_path) == 0:
                  f.write('Ticker,' + ','.join(dates_list) + '\n')

              values = []
              for date in dates_list:
                values.append(csv_data_dict[key].get(date, None))
              values_str = ','.join(map(str, values))
              f.write(f"{symbol},{values_str}\n")